In [7]:
import numpy as np
import tensorflow as tf
from othello import Board

boardDim = 8*8
dataType = tf.float32
hLayersDim = [128, 256, 128]

x = tf.placeholder(tf.int8, [None, boardDim])

layers = []
biases = []
activations = []

for i, dim in enumerate(hLayersDim):
    if i == 0:
        layers.append(tf.Variable("W"+str(i), dataType, [boardDim, dim]))
    elif i == len(hLayersDim) - 1:
        layers.append(tf.Variable("W"+str(i), dataType, [dim, boardDim]))
    else:
        layers.append(tf.Variable("W"+str(i), dataType, [hLayersDim[i-1], dim]))
    biases.append(tf.Variable("b"+str(i), dataType, [dim]))
    
#for i, dim in enumerate(hLayersDim):
#    activations.append(tf.nn.relu(bias, name=scope.name))
    